# I/O

*Notes from lecture on April 10th, 2017*

*Recall inter-subsystem architecture image*

Not only do devices need to be managed, but the buses need to be managed as well.

## Direct Memory Access (DMA)

- It is not efficient for the CPU to request data from I/O one byte as a time
- Direct Memory Access (DMA) controller has accecss to the system bus independent of the CPU

This is good!

- CPU is freed to pursue other tasks.
- I/O will interrupt when request completed.

Typical interaction:

1. OS owns bus
1. OS programs DMA (ON dev FROM addr TO addr, ACTION)
1. OS continues other processes
1. DMA waits for bus free
  1. DMA poll returns success
  1. DMA copies some data to internal buffer
  1. DMA relinquishes bus
  1. Loop until transfer complete
  1. Signal (interrupt)
1. OS acquires bus
1. OS copies data from DMA internal buffer

The OS needs to acknowledge the interrupt for this to work.  There are some intricacies to this ...

*Recall system bus image*

## Precise Interrupts

Modern hardware architectures execute many instructions concurrently.  Precise interrupts help synchronize events logically.  Things that need to happen area gauaranteed to have happened.  This makes handling interrupts much simpler.

### Properties of Precise Interrupts:

1. The program counter (PC) is saved in known place
1. All instructions before the one pointed by PC have fully executed
1. No instruction beyond the one pointed by PC have been executed
1. The execution state of the instruction pointed to by the PC is known.

> **Note**: this does not affect the scheduler.

## I/O Software

I/O software gives us the following properties:

- Device independence
- Uniform naming
- Error handling (better to handle errors closer to the source (device))
- Synchronous vs asynchronous (interrupt-driven)
- Buffering (lots of problems w/o buffering trigger heavy waiting (undesirable), think internet)
- Sharable vs. dedicated devices (network/disk vs. screen)

### Three Ways of Doing I/O

#### Programmed I/O

Here, CPU does all the work.  Think polling/busy-waiting scheme.  For example:

    copy_from_user(buffer, dev, count) {
        for(i = 0; i < count; ++i) {
            while(dev.status.rag != READY);
            copy(buffer[i] = dev.buffer);
        }
    }

#### Interrupt-driven I/O

Waiting for a device to be ready, the process is blocked and another process is scheduled.  When the device is ready it raises an interrupt.

#### Direct Memory Access (DMA)

DMA does the work instead of the CPU.  The CPU lets the DMA do its work and then interrupt.

> **Note**: DMA is, in fact, copying directly into DRAM.  Hence why it needs control of the bus during operation.
> **Note**: DMA does not always write to the same memory.  The OS provides locked pages for DMA to use.

## DMA Alternative Configurations

1. Single-bus, detached DMA
1. Single-bus, Integrated DMA I/O
1. Separate I/O bus

## OS Software Layers for I/O

- User-level I/O
- Device-independent operating system software
- Device drivers
- Interrupt handlers
- Hardware

> **Recall**: There are block devices and character devices.  Block=disk; Char=internet,keyboard

#### Device Drivers

- Device specific code for controlling the device
  - Read device registers from controller
  - Write device registers to issue commands
- Usually supplied by the device manufacturer
- Can be part of the kernel or at user-space (with system calls to access controller registers)
- OS defines a standard interface that drivers for block devices must follow and another standard for driver of character devices

- Main functions:
  - Receive abstract read/write from layer above and carry them out
  - Initialize the device
  - Log events
- ... plugged (oops!)

## Device Independent I/O Software

Use uniformm interfacing for device drivers

- Trying to make all devices look the same
- For each class of devices, the OS defines a set of functions that the driver must supply.
- This layer of OS maps symbolic device names onto proper drivers.

> **Note**: You can `cd /dev` to see a list of devices.  The naming convention corresponds to the device type and how you should attempt to interface with them!

## Device Intependent I/O Software

*Refer to diagam expressing the need for buffering (modem example)*

- Very inefficient

## Device Example: Clocks

### Hardware

Clocks are devices?  Who knew!  They change frequency many times/second to compensate load (CPU governer).

- Old style: tied to power line and causes interrupt on every voltage cycle
- New style: crystal oscillator + counter + holding register.  Clock decrements register until 0, then triggers interrupt

Typically a calibration period occurs, then interrupt signals passage of time (fixed interval, usually 4$\mu$s

> **Note**: This means there are usually 2 clocks, one constant and one variable.

> **Note**: Interrupts are the means for clock to "wake up" processor.  Think keypress to wake PC.  Same exact mechanism!

### Software

- Maintaining the time of the day
- Preventing processes from running longer than they are allowed to
- Accounting for CPU usage
- Handling alarm system call make by user prorcessis
- Providing watchdog timers for parts of the system itself
- Doing profiling, monitoring, and statistics gathering

> **Note**: Watchdog timers are highly privileged, need to be to detect when PC is hung!

## Device Example: Keyboards

- Contains simplified embedded processor that communicaties through a port with the controller at the motherboard.
- An interrupt is generated whenever a key is struck and a second one whenever a key is released.
- Keyboard driver extracts the information about what happens from the I/O port assigned to the keyboard.
- The number in the I/O port is called the scan code (7 bits for code + 1 bit for key press/release)

### Two Philosophies

1. Programs get a raw sequence of ASCII codes (raw mode, or noncanonical mode)
2. Driver handles all the intraline editing and just delivers corrected lines to the use programs (cooked mode or canonical mode)

> **Note**: Either way, a buffer is needed to store characters.
> **Note**: Keyboard buffer used to fill if you typed too quickly, "beeped" if it was full!

## Device Example: Mouse

Mouse only indicates changes in position not absolute position (`delta_x` and `delta_y`)

## I/O Software Layer Principals

Interrupts are facts of live, but should be hidden away, so that as little of the OS as possible knows about them.  The best way to hide interrupts is to have the driver starting an IO operation block until IO has completed and the interrupt occurs.  When interrupt happens, the interrupt handler handles the interrupt.  Once the handling of interrupt is done, the interrupt handler unblocks the device driver that started it.  This model works if drivers Re structured as kernel [rpcesses with their own states, stacks, and program counters.